### data ###

In [7]:
import zipfile
import os
import cv2

def unzip_file(zip_path, extract_to='.'):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

# Unzip manually downloaded files
unzip_file('WIDER_train.zip', 'wider_face/')
unzip_file('WIDER_val.zip', 'wider_face/')


In [8]:
from pathlib import Path

for split in ['train', 'val']:
    Path(f'datasets/face/images/{split}').mkdir(parents=True, exist_ok=True)
    Path(f'datasets/face/labels/{split}').mkdir(parents=True, exist_ok=True)


### **Dataset Preparation**

In [9]:
def process_wider_annotations(image_dir, annotation_file, output_label_dir, output_img_dir):
    # import os

    # Make output directories if they don't exist
    os.makedirs(output_label_dir, exist_ok=True)
    os.makedirs(output_img_dir, exist_ok=True)

    with open(annotation_file, 'r') as f:
        lines = f.readlines()

    idx = 0
    while idx < len(lines):
        filename = lines[idx].strip()
        idx += 1

        if idx >= len(lines):
            break  # avoid IndexError if file ends early

        try:
            num_faces = int(lines[idx].strip())
        except ValueError:
            print(f"⚠️ Skipped invalid line: {lines[idx].strip()}")
            break

        idx += 1

        image_path = os.path.join(image_dir, filename)
        label_path = os.path.join(output_label_dir, os.path.splitext(filename)[0] + '.txt')
        label_dir = os.path.dirname(label_path)
        os.makedirs(label_dir, exist_ok=True)

        # Copy image (optional)
        output_img_path = os.path.join(output_img_dir, filename)
        output_img_dirname = os.path.dirname(output_img_path)
        os.makedirs(output_img_dirname, exist_ok=True)
        if os.path.exists(image_path):
            cv2.imwrite(output_img_path, cv2.imread(image_path))
        else:
            print(f"🚫 Image not found: {image_path}")
            continue

        with open(label_path, 'w') as label_file:
            for _ in range(num_faces):
                if idx >= len(lines):
                    break  # avoid going out of range
                box = list(map(int, lines[idx].strip().split()))
                x, y, w, h = box[:4]
                cx = x + w / 2
                cy = y + h / 2
                label_file.write(f"0 {cx} {cy} {w} {h}\n")  # YOLO format: class x_center y_center width height
                idx += 1


# Run conversion
process_wider_annotations(
    image_dir='wider_face/WIDER_train/images',
    annotation_file='wider_face/wider_face_split/wider_face_train_bbx_gt.txt',
    output_label_dir='datasets/face/labels/train',
    output_img_dir='datasets/face/images/train'
)

process_wider_annotations(
    image_dir='wider_face/WIDER_val/images',
    annotation_file='wider_face/wider_face_split/wider_face_val_bbx_gt.txt',
    output_label_dir='datasets/face/labels/val',
    output_img_dir='datasets/face/images/val'
)


⚠️ Skipped invalid line: 0--Parade/0_Parade_Parade_0_630.jpg


In [10]:
yaml_content = """
path: datasets/face
train: images/train
val: images/val

nc: 1
names: ['face']
"""

with open("face.yaml", "w") as f:
    f.write(yaml_content)


In [11]:
from ultralytics import YOLO

# Load pretrained model
model = YOLO('yolov8n.pt')

# model.model.fuse()

# for param in list(model.model.parameters())[:-10]:
    # param.requires_grad = False


# Start training
model.train(
    data='face.yaml',
    epochs=15,
    imgsz=640,
    batch=16,
    name='yolov8_face'
)


New https://pypi.org/project/ultralytics/8.3.162 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.161 🚀 Python-3.12.3 torch-2.7.1+cu126 CPU (Intel Core(TM) i7-8665U 1.90GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=face.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_face9, nbs=64, nms=False, opset=None, optimize=False, o

train: Scanning /home/sayeed/Documents/Deep_learning/assignment-7/datasets/face/labels/train/0--Parade.cache... 280 images, 1 backgrounds, 279 corrupt: 100%|██████████| 280/280 [00:00<?, ?it/s]

train: /home/sayeed/Documents/Deep_learning/assignment-7/datasets/face/images/train/0--Parade/0_Parade_Parade_0_1014.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [127. 386.  10.  12. 252. 385.  12.  14.]
train: /home/sayeed/Documents/Deep_learning/assignment-7/datasets/face/images/train/0--Parade/0_Parade_Parade_0_1040.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [531.5 306.   75.  110. ]
train: /home/sayeed/Documents/Deep_learning/assignment-7/datasets/face/images/train/0--Parade/0_Parade_Parade_0_1042.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [  17.5  256.     5.     6.   313.   232.5   36.    41.   503.5  241.
   41.    50.   546.   220.    12.    14.   531.5  226.    11.    12.
  580.5  237.5   15.    17.   663.5  237.    13.    14.   703.   206.5
   12.    13.   697.5  231.5   11.    13.   677.   246.5   10.    15.
  731.5  247.    11.    18.   742.5  248.    15.    18.   769. 

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2904.1±744.0 MB/s, size: 229.3 KB)


/home/sayeed/Documents/Deep_learning/env/lib/python3.12/site-packages/torch/cuda/__init__.py:789: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/sayeed/Documents/Deep_learning/env/lib/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning /home/sayeed/Documents/Deep_learning/assignment-7/datasets/face/labels/val/0--Parade.cache... 3226 images, 0 backgrounds, 3222 corrupt: 100%|██████████| 3226/3226 [00:00<?, ?it/s]

...

                   all          4          4          0          0          0          0



15 epochs completed in 0.006 hours.
Optimizer stripped from runs/detect/yolov8_face9/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/yolov8_face9/weights/best.pt, 6.2MB

Validating runs/detect/yolov8_face9/weights/best.pt...
Ultralytics 8.3.161 🚀 Python-3.12.3 torch-2.7.1+cu126 CPU (Intel Core(TM) i7-8665U 1.90GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.44it/s]


                   all          4          4          0          0          0          0
Speed: 1.5ms preprocess, 80.5ms inference, 0.0ms loss, 9.2ms postprocess per image
Results saved to runs/detect/yolov8_face9


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7898ff25dc70>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [17]:
# from ultralytics import YOLO
import cv2
import random

# Load your fine-tuned model
model = YOLO("runs/detect/yolov8_face/weights/best.pt")

# Load the uploaded image
# image_path = "face_img.jpg"
# image = cv2.imread(image_path)

# Predict with low confidence threshold
# results = model(image_path, conf=0.1)[0]
results = model("datasets/face/images/val/0--Parade/0_Parade_marchingband_1_456.jpg", conf=0.05)[0]


# Show number of detected faces
print(f"Detected {len(results.boxes)} faces")

# Draw a random color box around each detected face
for box in results.boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    conf = float(box.conf[0])
    color = [random.randint(0, 255) for _ in range(3)]
    cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
    cv2.putText(image, f"{conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

# Save the output image
output_path = "predicted_faces.jpg"
cv2.imwrite(output_path, image)
print(f"✅ Saved output to: {output_path}")


FileNotFoundError: datasets/face/images/val/0--Parade/0_Parade_marchingband_1_456.jpg does not exist

In [16]:
print(f"Detected {len(results.boxes)} faces")


Detected 0 faces


In [14]:
# results = model("https://ultralytics.com/images/bus.jpg")

# annotated_img = results[0].plot()
# #
# plt.imsave("bus_detected.png", annotated_img)